<a href="https://colab.research.google.com/github/iranovianti/DP_Lab/blob/main/Interactive_visualization_multiple_HPLC_PDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title **INITIALIZE**
!git clone --quiet https://github.com/iranovianti/DP_Lab.git

import sys

sys.path.append('/content/DP_Lab')
import pda
import matplotlib.pyplot as plt
import glob
from ipywidgets import interact, widgets
from IPython.display import display

In [2]:
#@title **RUN VISUALIZATION**

def HPLC_list_from_PDA(pda_file_dict, wls=[280], gap=6e4, color='k', xlim=(0,15), xlabel='retention time (min)', rev=True, labels=None, wl_color=None):
  wls = pda.adjust_values(wls, pda.steps(200, 113, 4))
  pda_dict = {name: pda.PDA(filepath) for name, filepath in pda_file_dict.items()}
  samples = list(pda_dict.keys())

  if rev:
    samples = samples[::-1]

  plt.figure()
  pos = 0
  for i in range(len(samples)):
    sample = samples[i]
    c = color * len(wls)
    if wl_color:
      c = wl_color
    
    l = sample
    if labels:
      l = labels[pda_file_dict[sample]]
    
    rt = pda_dict[sample].t_step

    if len(wls) == 1:
      ch = pda_dict[sample].data.loc[:][wls[0]]
      y = ch.values + pos
      plt.plot(rt, y, c=c[0])
      plt.annotate(l, xy=(xlim[1], y[-1]), xytext=(xlim[1]+0.5, y[-1]))
    
    else:
      for ii in range(len(wls)):
        ch = pda_dict[sample].data.loc[:][wls[ii]]
        y = ch.values + pos
        plt.plot(rt, y, c=c[ii])
        if ii == 0:
          plt.annotate(l, xy=(xlim[1], y[-1]), xytext=(xlim[1]+0.5, y[-1]))
    
    pos += gap

  plt.gca().spines['top'].set_visible(False)
  plt.gca().spines['right'].set_visible(False)
  plt.gca().spines['left'].set_visible(False)

  plt.xlim(xlim)
  plt.yticks([])

  plt.xlabel('retention time (min)')
  plt.show()

#@markdown 1. Copy folder destination containing pda files (try copy /content/DP_Lab/sample_data as a start)
w_path = widgets.Text(value='', placeholder='Copy directory', description='Directory:', disabled=False)
#@markdown 2. Select multiple files using Ctrl + Click
selected_files = widgets.SelectMultiple(options=[''], rows=10, description='Files', disabled=False)

gap = widgets.FloatSlider(min=0.0, max=5e5, step=100, description='gap', orientation='vertical')
#@markdown 3. Wavelength value can bedirectly typed in the box
wavelengths = {'WL1': 208,
               'WL2': 256,
               'WL3': 280,
               'WL4': 360}

wl_widgets = {k: widgets.BoundedIntText(layout={'width': 'initial'}, value=v, min=200, max=648, step=1, description=k+':')\
              for k,v in wavelengths.items()}
wl_bool = {wl+'_bool': widgets.Checkbox(layout={'width': 'initial'}, value=False, description='')\
           for wl in wl_widgets.keys()}
wl_color = {wl+'_color': widgets.ColorPicker(concise=True, description='', value='black')\
            for wl in wl_widgets.keys()}

wl_params = wl_bool | wl_color
wl_selection = wl_widgets | wl_params

w_rev = widgets.Checkbox(value=False, description='Reverse order')

def display_files(*args):
  path = w_path.value + '/'
  filelist = [f.replace(path, '') for f in glob.glob(path+'*')]
  selected_files.options = filelist

w_path.observe(display_files, 'value')

def show_plot(x, y, z, r, **kwargs):
  pda_dict = {f'sample_{n}': x+'/'+fn for n,fn in enumerate(y)}
  wls = [kwargs[wl] for wl in wl_widgets.keys() if kwargs[wl+'_bool']]
  colors = [kwargs[wl+'_color'] for wl in wl_widgets.keys() if kwargs[wl+'_bool']]
  HPLC_list_from_PDA(pda_dict, gap=z, xlim=(0,15), rev=r, wls=wls, wl_color=colors)

ui = widgets.VBox([w_path, selected_files])
ui3 = widgets.VBox([widgets.HBox([wl_widgets[wl], wl_bool[wl+'_bool'], wl_color[wl+'_color']]) for wl in wl_widgets.keys()])
ui2 = widgets.VBox([ui3, w_rev])
combined = widgets.HBox([ui, ui2])

params = {'x':w_path, 'y':selected_files, 'z':gap, 'r':w_rev}
out = widgets.interactive_output(show_plot, params | wl_selection)

center = widgets.HBox([out, gap])
display(combined, center)